# LLM product description from keywords

Giskard is an open-source framework for testing all ML models, from LLMs to tabular models. Don’t hesitate to give the project a [star on GitHub](https://github.com/Giskard-AI/giskard) ⭐️ if you find it useful!

In this notebook, you’ll learn how to create comprehensive test suites for your model in a few lines of code, thanks to Giskard’s open-source Python library.

In this example, we illustrate the procedure using **OpenAI Client** that is the default one; however, please note that our platform supports a variety of language models. For details on configuring different models, visit our [🤖 Setting up the LLM Client page](../../sdk/index.rst)

In this tutorial we will walk through a practical use case of using the Giskard LLM Scan on a Prompt Chaining task, one step at a time. Given a product name, we will ask the LLM to process 2 chained prompts using `langchain` in order to provide us with a product description. The 2 prompts can be described as follows:

1. `keywords_prompt_template`: Based on the product name (given by the user), the LLM has to provide a list of five to ten relevant keywords that would increase product visibility.
2. `product_prompt_template`: Based on the given keywords (given as a response to the first prompt), the LLM has to generate a multi-paragraph rich text product description with emojis that is creative and SEO compliant.

Use-case:

* Two-step product description generation. 1) Keywords generation -> 2) Description generation;

Outline:

* Detect vulnerabilities automatically with Giskard's scan
* Automatically generate & curate a comprehensive test suite to test your model beyond accuracy-related metrics

### Install dependencies and setup notebook

Make sure to install the `giskard[llm]` flavor of Giskard, which includes support for LLM models.
Additionally, we will install `langchain` to define the prompt templates and pipeline logic.

In [ ]:
%pip install "giskard[llm]"  --upgrade
%pip install langchain langchain-community langchain-openai --upgrade

Now, let's import all of the libraries that we will use in the notebook.

In [1]:
import os
from operator import itemgetter

import openai
import pandas as pd
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

from giskard import Dataset, Model, scan


And, we set the OpenAI API key and display options.

In [ ]:
# Set the OpenAI API Key environment variable.
OPENAI_API_KEY = "..."
openai.api_key = OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Display options.
pd.set_option("display.max_colwidth", None)

Lastly, we define some of the constants that we will use in the notebook. 
Note that we are also defining two prompt templates that we will use to generate the keywords and the product description.

In [3]:
LLM_MODEL = "gpt-3.5-turbo"

TEXT_COLUMN_NAME = "product_name"

# First prompt to generate keywords related to the product name
KEYWORDS_PROMPT_TEMPLATE = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a helpful assistant that generate a CSV list of keywords related to a product name

    Example Format:
    PRODUCT NAME: product name here
    KEYWORDS: keywords separated by commas here

    Generate five to ten keywords that would increase product visibility. Begin!

    """,
        ),
        (
            "human",
            """
    PRODUCT NAME: {product_name}
    KEYWORDS:""",
        ),
    ]
)

# Second chained prompt to generate a description based on the given keywords from the first prompt
PRODUCT_PROMPT_TEMPLATE = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """As a Product Description Generator, generate a multi paragraph rich text product description with emojis based on the information provided in the product name and keywords separated by commas.

    Example Format:
    PRODUCT NAME: product name here
    KEYWORDS: keywords separated by commas here
    PRODUCT DESCRIPTION: product description here

    Generate a product description that is creative and SEO compliant. Emojis should be added to make product description look appealing. Begin!

    """,
        ),
        (
            "human",
            """
    PRODUCT NAME: {product_name}
    KEYWORDS: {keywords}
    PRODUCT DESCRIPTION:
        """,
        ),
    ]
)

## Detect vulnerabilities in your model

### Define a generation function

To run scans, we need to define a generation function that takes a dataframe as input and returns a list of product descriptions.
Using the prompt templates defined earlier we can create two `LLMChain` and concatenate them into a `SequentialChain` that takes as input the product name, and outputs a product description.

In [4]:
def generation_function(df: pd.DataFrame):
    llm = ChatOpenAI(temperature=0.2, model=LLM_MODEL)

    # Define the chains.
    keywords_chain = KEYWORDS_PROMPT_TEMPLATE | llm | StrOutputParser()
    product_description_chain = (
        {"keywords": keywords_chain, "product_name": itemgetter("product_name")}
        | PRODUCT_PROMPT_TEMPLATE
        | llm
        | StrOutputParser()
    )

    return [
        product_description_chain.invoke({"product_name": product_name})
        for product_name in df["product_name"]
    ]

### Wrap generation function as Giskard Model

Before running the automatic LLM scan, we need to wrap our model into Giskard's `Model` object.

In [5]:
# Wrap the description chain.
giskard_model = Model(
    model=generation_function,
    # A prediction function that encapsulates all the data pre-processing steps and that could be executed with the dataset
    model_type="text_generation",  # Either regression, classification or text_generation.
    name="Product keywords and description generator",  # Optional.
    description="Generate product description based on a product's name and the associated keywords."
    "Description should be using emojis and being SEO compliant.",  # Is used to generate prompts
    feature_names=["product_name"],  # Default: all columns of your dataset.
)

2025-09-11 14:58:39,931 pid:80445 MainThread giskard.models.automodel INFO     Your 'prediction_function' is successfully wrapped by Giskard's 'PredictionFunctionModel' wrapper class.


### Test your model with a Giskard Dataset

We can also optionally create a small dataset of queries to test that the model wrapping worked.
Let’s check that the model is correctly wrapped by running it on a small dataset of sample input prompts:

In [6]:
# Optional: Wrap a dataframe of sample input prompts to validate the model wrapping and to narrow specific tests' queries.
corpus = [
    "Double-Sided Cooking Pan",
    "Automatic Plant Watering System",
    "Miniature Exercise Equipment",
]

giskard_dataset = Dataset(pd.DataFrame({TEXT_COLUMN_NAME: corpus}), target=None)

# Validate the wrapped model and dataset.
giskard_model.predict(giskard_dataset).prediction

2025-09-11 14:58:43,402 pid:80445 MainThread giskard.datasets.base INFO     Your 'pandas.DataFrame' is successfully wrapped by Giskard's 'Dataset' wrapper class.
2025-09-11 14:58:43,413 pid:80445 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'product_name': 'object'} to {'product_name': 'object'}
2025-09-11 14:58:57,436 pid:80445 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (3, 1) executed in 0:00:14.032847


array(["🌟 Introducing the innovative Double-Sided Cooking Pan! 🍳✨ This kitchen essential is a game-changer for all cooking enthusiasts out there. 🍽️🔥\n\n👩\u200d🍳 With its non-stick surface, this versatile pan is perfect for all your cooking needs. 🥘 Whether you're grilling, frying, or sautéing, this dual-purpose pan has got you covered! 🍳🥩\n\n💫 The reversible design of this pan makes it a multi-functional gem in your kitchen. 🔁 From a double-sided grill to a flat cooking surface, the possibilities are endless! 🔄🍳\n\n🔥 Made from heat-resistant materials, this pan can withstand high temperatures, ensuring even cooking every time. 🌡️ And the best part? It's super easy to clean, making your post-cooking cleanup a breeze! 🧼🪠\n\n👨\u200d🍳 Elevate your cooking game with the Double-Sided Cooking Pan - the ultimate cooking tool for every home chef! 🌟🍳✨",
       'PRODUCT NAME: Automatic Plant Watering System\nKEYWORDS: smart irrigation, garden watering system, plant care, automated watering, indo

### Scan your model for vulnerabilities with Giskard

We can now run Giskard's `scan` to generate an automatic report about the model vulnerabilities. This will thoroughly test different classes of model vulnerabilities, such as harmfulness, hallucination, prompt injection, etc.

The scan will use a mixture of tests from predefined set of examples, heuristics, and LLM based generations and evaluations.

Since running the whole scan can take a bit of time, let’s start by limiting the analysis to the hallucination category:

In [ ]:
results = scan(giskard_model)

You can save and share them with your team using the `to_html` and `to_json` methods but for now we will just display the results:

In [8]:
display(results)

<iframe id="scan-13869046608" srcdoc="<!doctype html>
<html lang="en">

<head>
 <title>Giskard Scan Results</title>
 <base target="_blank">
 <meta charset="utf-8">
 
<style>pre code.hljs{display:block;overflow-x:auto;padding:1em}code.hljs{padding:3px 5px}
/*!
 Theme: GitHub Dark
 Description: Dark theme as seen on github.com
 Author: github.com
 Maintainer: @Hirse
 Updated: 2021-05-15

 Outdated base version: https://github.com/primer/github-syntax-dark
 Current colors taken from GitHub's CSS
*/.hljs{background:#0d1117;color:#c9d1d9}.hljs-doctag,.hljs-keyword,.hljs-meta .hljs-keyword,.hljs-template-tag,.hljs-template-variable,.hljs-type,.hljs-variable.language_{color:#ff7b72}.hljs-title,.hljs-title.class_,.hljs-title.class_.inherited__,.hljs-title.function_{color:#d2a8ff}.hljs-attr,.hljs-attribute,.hljs-literal,.hljs-meta,.hljs-number,.hljs-operator,.hljs-selector-attr,.hljs-selector-class,.hljs-selector-id,.hljs-variable{color:#79c0ff}.hljs-meta .hljs-string,.hljs-regexp,.hljs-string{color:#a5d6ff}.hljs-built_in,.hljs-symbol{color:#ffa657}.hljs-code,.hljs-comment,.hljs-formula{color:#8b949e}.hljs-name,.hljs-quote,.hljs-selector-pseudo,.hljs-selector-tag{color:#7ee787}.hljs-subst{color:#c9d1d9}.hljs-section{color:#1f6feb;font-weight:700}.hljs-bullet{color:#f2cc60}.hljs-emphasis{color:#c9d1d9;font-style:italic}.hljs-strong{color:#c9d1d9;font-weight:700}.hljs-addition{background-color:#033a16;color:#aff5b4}.hljs-deletion{background-color:#67060c;color:#ffdcd7}.hljs-copy-wrapper{overflow:hidden;position:relative}.hljs-copy-button:focus,.hljs-copy-wrapper:hover .hljs-copy-button{transform:translateX(0)}.hljs-copy-button{background-color:#2d2b57;background-color:var(--hljs-theme-background);background-image:url('data:image/svg+xml;charset=utf-8,<svg xmlns="http://www.w3.org/2000/svg" width="16" height="16" fill="none" viewBox="0 0 24 24"><path fill="%23fff" fill-rule="evenodd" d="M6 5a1 1 0 0 0-1 1v14a1 1 0 0 0 1 1h12a1 1 0 0 0 1-1V6a1 1 0 0 0-1-1h-2a1 1 0 1 1 0-2h2a3 3 0 0 1 3 3v14a3 3 0 0 1-3 3H6a3 3 0 0 1-3-3V6a3 3 0 0 1 3-3h2a1 1 0 0 1 0 2H6Z" clip-rule="evenodd"/><path fill="%23fff" fill-rule="evenodd" d="M7 3a2 2 0 0 1 2-2h6a2 2 0 0 1 2 2v2a2 2 0 0 1-2 2H9a2 2 0 0 1-2-2V3Zm8 0H9v2h6V3Z" clip-rule="evenodd"/></svg>');background-position:50%;background-repeat:no-repeat;border:1px solid #ffffff22;border-radius:.25rem;color:#fff;height:2rem;position:absolute;right:1em;text-indent:-9999px;top:1em;transition:background-color .2s ease,transform .2s ease-out;width:2rem}.hljs-copy-button:hover{border-color:#ffffff44}.hljs-copy-button:active{border-color:#ffffff66}.hljs-copy-button[data-copied=true]{background-image:none;text-indent:0;width:auto}@media (prefers-reduced-motion){.hljs-copy-button{transition:none}}.hljs-copy-alert{clip:rect(0 0 0 0);-webkit-clip-path:inset(50%);clip-path:inset(50%);height:1px;overflow:hidden;position:absolute;white-space:nowrap;width:1px}
/*! tailwindcss v3.3.2 | MIT License | https://tailwindcss.com*/*,:after,:before{border:0 solid #e5e7eb;box-sizing:border-box}:after,:before{--tw-content:""}html{-webkit-text-size-adjust:100%;font-feature-settings:normal;font-family:ui-sans-serif,system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji;font-variation-settings:normal;line-height:1.5;-moz-tab-size:4;-o-tab-size:4;tab-size:4}body{line-height:inherit;margin:0}hr{border-top-width:1px;color:inherit;height:0}abbr:where([title]){-webkit-text-decoration:underline dotted;text-decoration:underline dotted}h1,h2,h3,h4,h5,h6{font-size:inherit;font-weight:inherit}a{color:inherit;text-decoration:inherit}b,strong{font-weight:bolder}code,kbd,pre,samp{font-family:ui-monospace,SFMono-Regular,Menlo,Monaco,Consolas,Liberation Mono,Courier New,monospace;font-size:1em}small{font-size:80%}sub,sup{font-size:75%;line-height:0;position:relative;vertical-align:baseline}sub{bottom:-.25em}sup{top:-.5em}table{border-collapse:co

## Generate comprehensive test suites automatically for your model

### Generate test suites from the scan

The objects produced by the scan can be used as fixtures to generate a test suite that integrates all detected vulnerabilities. Test suites allow you to evaluate and validate your model's performance, ensuring that it behaves as expected on a set of predefined test cases, and to identify any regressions or issues that might arise during development or updates.

In [9]:
test_suite = results.generate_test_suite("Test suite generated by scan")
test_suite.run()

2025-09-11 15:14:21,709 pid:80445 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'product_name': 'object'} to {'product_name': 'object'}
2025-09-11 15:14:21,711 pid:80445 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (10, 1) executed in 0:00:00.026927
2025-09-11 15:14:21,715 pid:80445 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'product_name': 'object'} to {'product_name': 'object'}
2025-09-11 15:14:21,717 pid:80445 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (10, 1) executed in 0:00:00.005305
Executed 'Basic Sycophancy' with arguments {'model': <giskard.models.function.PredictionFunctionModel object at 0x10b3ab650>, 'dataset_1': <giskard.datasets.base.Dataset object at 0x32a59f080>, 'dataset_2': <giskard.datasets.base.Dataset object at 0x32a59f4d0>}: 
               Test failed
               Metric: 2
               
               
2025-09-11 15:14:30,537 pid:8044

<TestSuiteResult (failed)>